## Training a PyTorch Model

This tutorial shows how to train a Logistic Regression model in PyTorch using the Census API's `experimental.ml.ExperimentDataPipe` class. This is intended only to demonstrate the use of the `ExperimentDataPipe`, and not as an example of how to train a biologically useful model.

This tutorial assumes a basic familiarity with PyTorch and the Census API. See the [Querying and fetching the single-cell data and cell/gene metadata](https://chanzuckerberg.github.io/cellxgene-census/notebooks/api_demo/census_query_extract.html) notebook tutorial for a quick primer on Census API usage.

**Contents**
* [Open the Census](#Open-the-Census)
* [Create a DataLoader](#Create-a-DataLoader)
* [Define the model](#Define-the-model)
* [Train the model](#Train-the-model)
* [Make predictions with the model](#Make-predictions-with-the-model)


## Open the Census

First, obtain a handle to the Census data, in the usual manner:

In [1]:
import cellxgene_census

census = cellxgene_census.open_soma()

The "stable" release is currently 2023-05-15. Specify 'census_version="2023-05-15"' in future calls to open_soma() to ensure data consistency.


## Create a DataLoader

To train a model in PyTorch using this `census` data object, first instantiate an `ExperimentDataPipe` as follows:

In [2]:
import cellxgene_census.experimental.ml as census_ml
import tiledbsoma as soma

experiment = census["census_data"]["homo_sapiens"]

experiment_datapipe = census_ml.ExperimentDataPipe(
    experiment,
    measurement_name="RNA",
    X_name="raw",
    obs_query=soma.AxisQuery(value_filter="tissue_general == 'tongue' and is_primary_data == True"),
    obs_column_names=["cell_type"],
    batch_size=16,
)

This class provides an implementation of PyTorch's [`DataPipe` interface](https://pytorch.org/data/main/torchdata.datapipes.iter.html), which defines a common mechanism for wrapping and accessing training data from any underlying source. The `ExperimentDataPipe` class encapsulates the details of querying and retrieving Census data from a single SOMA `Experiment` and returning it to the caller as PyTorch Tensors. Most importantly, it retrieves the data lazily from the Census in batches, avoiding having to load the entire training dataset into memory at once.

The constructor only requires a single parameter, `experiment`, which is a `soma.Experiment` containing the data of the organism to be used for training.

To retrieve a subset of the Experiment's data, along either the `obs` or `var` axes, you may specify query filters via the `obs_query` and `var_query` parameters, which are both `soma.AxisQuery` objects.

The values for the prediction label(s) that you intend to use for training are specified via the `obs_column_names` array. 

Finally, the `batch_size` allows you to specify the number of obs rows (cells) to return in each training iteration step. You may exclude this parameter if you want single rows (`batch_size=1`).

(Note: PyTorch also provides `DataSet` as a legacy interface for wrapping and accessing training data sources, but a `DataPipe` can be used interchangeably.)

You can inspect the shape of the full dataset, without causing the full dataset to be loaded:

In [3]:
experiment_datapipe.shape

(15020, 60664)

### Splitting the dataset

You may split the overall dataset into the typical training, validation, and test sets by using the PyTorch [`RandomSplitter`](https://pytorch.org/data/main/generated/torchdata.datapipes.iter.RandomSplitter.html#torchdata.datapipes.iter.RandomSplitter) `DataPipe`. Using PyTorch's functional form for chaining `DataPipe`s, this is done as follows:

In [4]:
train_datapipe, validation_datapipe, test_datapipe = experiment_datapipe.random_split(
    weights={"train": 0.8, "validation": 0.1, "test": 0.1}, seed=1
)

### Shuffling the training dataset

Commonly, you will want to randomize the ordering of the training data for each training epoch. You can use PyTorch [`Shuffler`](https://pytorch.org/data/main/generated/torchdata.datapipes.iter.Shuffler.html) `DataPipe`, chaining it to the earlier `experiment_dataloader` `DataPipe`. Again, using PyTorch's functional form for chaining the `Datapipe`s, we have:

In [5]:
shuffled_train_datapipe = train_datapipe.shuffle()

Note that `shuffle()` operates on a fixed size of training rows, which is 10,000 by default, and can be specified via the `buffer_size` param. Increasing this value may be useful for improving the heterogeneity of cells appearing within a given shuffling "window", as low heterogeneity may result in overfitting in the model. Note, however, that larger `buffer_size` values will require more memory. 

If the training data is known to fit fully within in available memory, you can shuffle the entire dataset using `experiment_datapipe.shuffle(len(experiment_datapipe))`.

### Create the DataLoader

With the full set of DataPipe operations chained together, we can now instantiate a PyTorch [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) on the training data. 

In [6]:
experiment_dataloader = census_ml.experiment_dataloader(shuffled_train_datapipe)

Alternately, you can instantiate a `DataLoader` object directly via its constructor. However, many of the parameters are not usable with iterable-style DataPipes, which is the case for `ExperimentDataPipe`. In particular, the `batch_size`, `sampler`, `batch_sampler`, `collate_fn` parameters should not be specified. Using `experiment_dataloader` helps enforce correct usage.

## Define the model

With the training data retrieval code now in place, we can move on to defining a simple logistic regression model, using PyTorch's `torch.nn.Linear` class:

In [7]:
import numpy as np
import torch


class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

Next, we define a function to train the model for a single epoch:

In [8]:
def train_epoch(model, train_dataloader, loss_fn, optimizer, device):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, y_batch = batch

        X_batch = X_batch.to(device)

        # Perform prediction
        outputs = model(X_batch)

        # Determine the predicted label
        probabilities = torch.nn.functional.softmax(outputs, 1)
        predictions = torch.argmax(probabilities, axis=1).cpu()

        # Compute the loss and perform back propagation

        # Exclude the cell_type labels, which are in the second column
        y_batch = y_batch[:, 1]
        y_batch = y_batch.to(device).cpu()

        train_correct += (predictions == y_batch).sum().item()
        train_total += len(predictions)

        loss = loss_fn(outputs, y_batch.long())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_loss /= train_total
    train_accuracy = train_correct / train_total
    return train_loss, train_accuracy

Note the line, `X_batch, y_batch = batch`. Since the `train_dataloader` was configured with `batch_size=16`, these variables will hold tensors of rank 2. The `X_batch` tensor will appear, for example, as:

```
tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 2.,  ..., 0., 3., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 8.]])
      
```

For `batch_size=1`, the tensors will be of rank 1. The `X_batch` tensor will appear, for example, as:

```
tensor([0., 0., 0.,  ..., 1., 0., 0.])
```
    
Secondly, note the line, `y_batch = y_batch[:, 1]`. This line is extracting the user-specified `obs` `cell_type` training labels from the second column of the `y_batch` rank 2 Tensor.  For example, this would take a `y_batch` tensor that looks like:
```
tensor([[42496620,        1],
        [42496621,        1],
        [42496622,        3],
        ...,
        [42496633,        2],
        [42496634,        1],
        [42496635,        4]], dtype=torch.int32)
      
```
and return:
```
tensor([1, 1, 3, ..., 2, 1, 4])

```
Note that cell type values are integer-encoded values, which can be decoded using `experiment_datapipe.obs_encoders()` (more on this below).


## Train the model

Finally, we are ready to train the model. Here we instantiate the model, a loss function, and an optimization method and then iterate through the desired number of training epochs. Note how the `train_dataloader` is passed into `train_epoch`, where for each epoch it will provide a new iterator through the training dataset.

In [9]:
RANDOM_SEED = 12345
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# The size of the input dimension is the number of genes
input_dim = experiment_datapipe.shape[1]

# The size of the output dimension is the number of distinct cell_type values
cell_type_encoder = experiment_datapipe.obs_encoders()["cell_type"]
output_dim = len(cell_type_encoder.classes_)

model = LogisticRegression(input_dim, output_dim).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for epoch in range(5):
    train_loss, train_accuracy = train_epoch(model, experiment_dataloader, loss_fn, optimizer, device)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.7f} Accuracy {train_accuracy:.4f}")

Epoch 1: Train Loss: 0.1156322 Accuracy 0.4494
Epoch 2: Train Loss: 0.1095988 Accuracy 0.7357
Epoch 3: Train Loss: 0.1089615 Accuracy 0.7300
Epoch 4: Train Loss: 0.1080885 Accuracy 0.7275
Epoch 5: Train Loss: 0.1101440 Accuracy 0.4220


## Make predictions with the model

To make predictions with the model, we first create a new `DataLoader` using the `test_datapipe`, which provides the "test" split of the original dataset. For this example, we will only make predictions on a single batch of data from the test split.

In [10]:
experiment_dataloader = census_ml.experiment_dataloader(test_datapipe)

X_batch, y_batch = next(iter(experiment_dataloader))

Next, we invoke the model on the `X_batch` input data and extract the predictions:

In [11]:
model.eval()

outputs = model(X_batch)

probabilities = torch.nn.functional.softmax(outputs, 1)
predictions = torch.argmax(probabilities, axis=1).cpu()

predictions

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

The predictions are returned as the encoded values of `cell_type` label. To recover the original cell type labels as strings, we decode using the `cell_type_encoder`. Here, we obtain the encoder from `experiment_datapipe.obs_encoders()`.

At inference time, if the model inputs are not obtained via an `ExperimentDataPipe`, one could pickle the encoder at training time and save it along with the model. Then, at inference time it can be unpickled and used as shown below.

In [12]:
cell_type_encoder = experiment_datapipe.obs_encoders()["cell_type"]

predicted_cell_types = cell_type_encoder.inverse_transform(predictions)

predicted_cell_types

array(['basal cell', 'basal cell', 'basal cell', 'basal cell',
       'basal cell', 'basal cell', 'basal cell', 'basal cell',
       'basal cell', 'basal cell', 'basal cell', 'basal cell',
       'basal cell', 'basal cell', 'basal cell', 'basal cell'],
      dtype=object)

Finally, we create a Pandas DataFrame to examine the predictions:

In [13]:
import pandas as pd

pd.DataFrame(
    {
        "actual cell type": cell_type_encoder.inverse_transform(y_batch[:, 1].numpy()),
        "predicted cell type": predicted_cell_types,
    }
)

,actual cell type,predicted cell type
0,epithelial cell,basal cell
1,epithelial cell,basal cell
2,epithelial cell,basal cell
3,epithelial cell,basal cell
4,epithelial cell,basal cell
5,epithelial cell,basal cell
6,epithelial cell,basal cell
7,epithelial cell,basal cell
8,epithelial cell,basal cell
9,epithelial cell,basal cell


As we can see, the predictions are poor. It is as left as an excercise to the reader to find a better performing model.